Cargamos las librerías, el conjunto de datos y algunas variables que emplearemos durante el proceso:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pickle
%matplotlib inline

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

df = pickle.load(open('../processed_data/houses_cleaned.p',"rb"))

# Omitimos los warnings
import warnings
warnings.filterwarnings('ignore')

# Modelo de regresion logística

El primer modelo que implementaremos será un modelo de regresión logística, con el cual, trataremos de predecir si una vivienda cuesta más o menos que cierta cantidad. 

Así, podríamos utilizar como umbral, por ejemplo, el crédito que disponemos para comprar nuestra casa. Con lo cual, una posible aplicación de este modelo podría ser la de discernir si una casa se ajusta o no a nuestro presupuesto con tan solo indicarle las características deseadas (metros cuadrados, número de habitaciones, ubicación, ...).

## Preparación de los datos

Los modelos de regresión requieren que las variables sean numéricas. Por lo tanto, para poder incluir variables categóricas, tenemos que realizar un **one-hot-encoding**. En nuestro conjunto de datos tenemos las siguientes variables categóricas:

+ __<font color=#5F66A1>house_type</font>__
+ __<font color=#5F66A1>condition</font>__
+ __<font color=#5F66A1>garage</font>__
+ __<font color=#5F66A1>location</font>__
+ __<font color=#5F66A1>loc_city</font>__
+ __<font color=#5F66A1>loc_zone</font>__

En el caso de las variables relacionadas con la ubicación, en realidad las 3 hacen referencia a una sola, solo que con granularidad distinta. Por ello, optaremos por quedarnos con la más específica, __<font color=#5F66A1>location</font>__, y descartaremos el resto. Por tanto, procedemos a crear las variables *dummy*:

In [ ]:
dummy_houseType = pd.get_dummies(df['house_type'])
dummy_garage = pd.get_dummies(df['garage'], prefix='garage')
dummy_condition = pd.get_dummies(df['condition'])
dummy_location = pd.get_dummies(df['location'])
dummy_houseType.head(5)

Por cada variable, hemos generado tantas nuevas variables como categorías distintas hubiere. Estas nuevas variables son binarias, y toman como valor un 1 solo si la vivienda es de dicha categoría, tomando como valor 0 en el resto de casos. Puede comprobarse lo descrito en el ejemplo superior, donde vemos que, por cada tipo de casa, se ha generado una nueva variable binaria que toma como valor 1 solo si la vivienda corresponde con su categoría.

Por otro lado, en los modelos de regresión conviene no emplear variables correlacionadas, ya que su inclusion genera multicolinealidad, lo cual es un factor de inestabilidad que ha de evitarse siempre que sea posible. Sin embargo, al crear las variables *dummy*, hemos añadido cierto nivel de multiconlinealidad, ya que las variables no son independientes entre sí. Esto se percibe claramente al pensar que no hace falta la variable **Ático** para saber cuando una vivienda es un ático. Basta con ver cuando el resto de variables tiene todas valores nulos para deducir que se trata de un ático.

No obstante, esta multicolinealidad la podemos evitar fácilmente desprendiéndonos de una de las variables generadas. Así, procedemos a descartar una columna por cada variable dummy que hemos creado. En el caso arriba descrito del tipo de vivienda, por ejemplo, descartaremos la columna *Otros*, ya que es la menos representativa:

In [ ]:
dummy_houseType = dummy_houseType.drop(columns=['Otros'])
dummy_garage = dummy_garage.drop(columns=['garage_sin garaje'])
dummy_condition = dummy_condition.drop(columns=['buen estado'])
dummy_location = dummy_location.iloc[:,1:]
dummy_houseType.head(5)

Una vez tenemos ya nuestras variables codificadas, solo nos resta añadirlas a nuestro conjunto de datos principal, a la par que nos deshacemos de todas las variables categóricas aún sin codificar: 

In [ ]:
df_dummy = (df
            .join([dummy_houseType, dummy_garage, dummy_condition, dummy_location])
            .drop(columns=['house_type', 'garage', 'condition', 'location', 'loc_zone', 'loc_city']))

A continuación, descartamos tanto las variables de identificación y exploración (**<font color=#5F66A1>house_id</font>** y **<font color=#5F66A1>ad_description</font>**), así como las variables que en el análisis exploratorio hemos descubierto están correlacionadas entre sí (**<font color=#5F66A1>pop_total</font>** y **<font color=#5F66A1>pop_>=65</font>**)

In [ ]:
df_pred = df_dummy.drop(columns=['house_id', 'ad_description', 'pop_total', 'pop_>=65'])

Por último generamos nuestra variable objetivo binaria, a la cual denominaremos **credit**, y que tomará como valor *True* en las viviendas que tengan precio superior a nuestro crédito, y *False* en caso contrario.

Asimismo, estableceremos nuestro crédito en 230.000€:

In [ ]:
credit = 230000
df_pred["credit"] = np.where(df["price"] < credit, True, False)

perc = round((100*df_pred.credit.value_counts()[True])/len(df_pred),2)
print(f'Porcentaje de datos con precio inferior al crédito: {perc}%')

## Implementación del modelo

### Division del conjunto de datos

El primer paso para implementar nuestro modelo, será dividir nuestro conjunto de datos en 2 subconjuntos, uno de entrenamiento, y otro de test, en una proporción de 75% y 25% respectivamente. Por otro lado, separaremos también nuestra variable dependiente (X e y):

In [ ]:
X = df_pred.drop(columns='credit')
y = df_pred.credit

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.25, random_state=89, shuffle=True)

print(f'Número de datos para entrenamiento: {len(X_train)}')
print(f'Número de datos para test: {len(X_test)}')

### Predicciones

Una vez disponemos ya de los datos debidamente preparados, procedemos a entrenar nuestro modelo logístico con el subcojunto de *train*, tras lo cual, lo validaremos haciendo predicciones sobre el conjunto de *test*. Finalmente, además de comprobar la precision (*accuracy*) de nuestro modelo, visualizaremos la curva ROC.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

modelo = LogisticRegression(random_state=1)
modelo.fit(X_train, y_train)
y_pred  =  [x[1] for x in modelo.predict_proba(X_test)]

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
acc = 100*round(metrics.accuracy_score(y_test, [1 if y > 0.5 else 0 for y in y_pred]),4)

print(f'\nAccuracy: {acc} %\n')

plt.figure(figsize=(12, 5))
plt.plot(fpr, tpr, linewidth=2, color='#e9b49e')
plt.title('ROC CURVE')
plt.text(0.1, 0.8, f'AUC: {round(metrics.auc(fpr, tpr),4)}'
         , size='large', color='#a95f81', weight='semibold')
plt.show()

Vemos que el modelo que hemos implementado es capaz de determinar de forma acertada si una vivienda se ajusta o no a nuestro presupuesto en algo más de 9 de cada 10 ocasiones, lo cual no está nada mal. Quizás una elección de algún otro modelo más complejo incluso pudiera mejorar esta precisión. No obstante, dejaremos este modelo así, y nos centraremos, sobre todo, en la implementación del siguiente modelo de regresión que predice el precio de las viviendas.

# Modelo de regresión: predicción del precio de las viviendas

El segundo modelo que implementaremos corresponde con un modelo de regresión con el que trataremos de predecir el precio de las viviendas a partir de sus características.

## XG-Boost

Para el caso del segundo modelo de regresión, nos valdremos de la técnica de potenciación del gradiente (*Gradient Boosting*), la cual sirve para generar un modelo predictivo de gran precisión mediante la suma de muchos modelos de predicción débiles (árboles de decisión).

Más concretamente, emplearemos una implementación de esta técnica llamada **XG-Boost**, *eXtreme Gradient Boosting*, ya que se ha convertido, gracias a su eficiencia y eficacia, en uno de los modelos más empleados en la actualidad para lidiar con problemas de regresión y clasificación relacionados con datos tabulares, como en este caso.

Este gran rendimiento viene dado, entre otros factores, por el amplio abanico de hiperparámetros que contiene. Estos hiperparámetros permiten un ajuste muy minucioso y, con una buena afinación, se puede lograr optimizar y mejorar el resultado obtenido de forma considerable.

No obstante, esta afinación no es trivial, y requiere que, para conseguir buenos resultados, sea necesaria una cuidadosa estrategia de hiperparametrización.

Así, dado que gran parte de la implementación de nuestro modelo de regresión recae sobre esta búsqueda, conviene realizar una breve descripción inicial acerca de cuales son estos hiperparámetros, y de qué modo afectan a nuestro modelo:

### Hiperparámetros de XG-Boost

+ __<font color=#5F66A1>objective</font>__: 

Con este hiperparámetro especificamos qué tipo de modelo queremos construir. En nuestro caso, el modelo será el de regresión por mínimos cuadráticos ("reg:squarederror").


+ __<font color=#5F66A1>eval_metric</font>__:

Con este hiperparámetro especificamos la función de coste que tratamos de minimizar. Así, cada nuevo árbol que XG-Boost genere, tendrá por único objetivo minimizar esta función.

La función de coste que nosotros emplearemos será la raíz del error cuadrático medio (**RMSE**). Por lo tanto, cuanto menor sea el **RMSE** que obtengamos con un modelo, asumiremos que mejor es éste, ya que, en suma, los precios reales de las viviendas se ajustarán más a su hiperplano de regresión que al de otros modelos.

Aún así, si bien para nuestro escenario hemos decidido que *RMSE* parece la función de coste más apropiada, dependiendo de lo que se quisieramos obtener, podríamos haber optado por utilizar otra función de coste, como por ejemplo *MAE*, *LOG COSH* o *HUBER*.


+ __<font color=#5F66A1>n_estimators</font>__:

Sirve para establecer cuántos árboles queremos exactamente que se construyan. Este hiperparámetro juega un papel fundamental en el aprendizaje de nuestro modelo, ya que, con cada nuevo árbol, XG-Boost minimizara un poco más el RMSE. Por tanto, cuanto mayor sea el número de árboles que empleemos, podemos afirmar que más estaremos minimizado nuestro error...

...no obstante, resulta necesario subrayar aquí que el error que XG-Boost minimizará será el error del conjunto de entrenamiento. Es decir, el modelo aprenderá a predecir cada vez más certeramente el precio de aquellas viviendas con las que se entrene. Pero el hecho de que aprenda a predecir cada vez mejor los precios de estas casas, no siempre conlleva que aprenda a hacer lo propio con otras viviendas. Es más, lo que sucederá es que, a partir de un cierto número de árboles, el modelo se sobrespecializará tanto (*overfitting*) en los datos de entrenamiento que ya no sabrá generalizar este conocimiento, y empezará a realizar predicciones cada vez más desacertadas. Por ello, la clave, aquí, será ajustar este hiperparámetro hasta encontrar ese punto en el cual se consigue un modelo complejo, pero que además es capaz de generalizar.


+ __<font color=#5F66A1>early_stopping</font>__:

Una buena estrategia para obtener el número de árboles adecuado es dividir el conjunto de entrenamiento en 2 nuevos subconjuntos, y usar uno para entrenar el modelo, y otro para validarlo.

De este modo, escogiendo un número muy elevado de árboles, podremos ver que tanto el error en el conjunto de entrenamiento, como en el de validación, en ambos, va disminuyendo. Sin embargo, veremos que, llegados a un punto, pese a que el error en el conjunto entrenamiento sigue disminuyendo, el error en el conjunto de validación ya no disminuye más, sino que empieza paulatinamente a incrementar.

Ese punto de inflexión será el punto en el cual el modelo se está empezando a sobrespecializar. Por lo tanto, nos convendrá seleccionar el número de árboles justo previo a ese punto.

Para ello, tenemos diferentes alternativas. La forma más sencilla sería la de entrenar nuestro modelo con un número muy elevado de árboles, y evaluar a posteriori cuándo se ha dado el punto de inflexión. Sin embargo, entrenar un modelo con un número muy elevado de árboles es costoso. Por ello, una forma más eficiente que esa puede ser la de ir monitorizando la tendencia del error en el conjunto de validación, y detener la creación de árboles en cuanto se detecte que éste ya no decrece.

Pero, dado que el modelo no busca minimizar el error del conjunto de validación, puede ocurrir que se de un pequeño paso en falso en algún momento que haga aumentar por un momento el error, pero que no por ello hayamos llegado al punto donde empieza a generarse *overfitting*. Es más, estos pequeños retrocesos en el error del conjunto de validación son muy habituales a partir de cierto punto.

Con lo cual, un enfoque aún mejor que el de parar el crecimiento al primer indicio, puede ser el de detenerse solo tras un cierto número de pasos en falso consecutivos. Y eso es para lo que sirve exactamente el hiperparámetro __early_stopping__; para controlar cuántos pasos en falso podemos dar antes de considerar que la tendencia no es reversible, y que es mejor dejar ya de generar más árboles, evitando de tal modo el tener que continuar con un costoso entrenamiento en vano.


+ __<font color=#5F66A1>learning_rate</font>__:

Se ha mencionado ya en varias ocasiones que XG-Boost generará nuevos árboles, mediante los cuales, buscará minimizar el error cuadrático medio. Sin embargo, hasta ahora no hemos hablado de la velocidad con la que XG-Boost trata de acercarse a este mínimo. Este hiperparámetro sirve precisamente para eso, para controlar la tasa de aprendizaje con la cual un árbol nuevo trata de corregir los errores que comete.

Cuanto mayor sea la tasa, más rápido tratará el modelo de alcanzar ese mínimo. Esto será muy beneficioso en las primeras iteraciones, ya que con pocos pasos, pasaremos de tener un modelo con un RMSE altísimo, a otro con un RMSE bastante cerca del mínimo. Sin embargo, cerca de ese mínimo será cuando se requiera que cada ajuste sea cada más y áms sutil. Por ello, si nuestra tasa de aprendizaje es muy alta, solo será capaz de generar cambios bruscos, y nunca podrá converger lo suficiente, dando como resultado un RMSE más alto que el que obtendríamos de haber optado por una tasa inferior.

Por otro lado, eligiendo tasas muy pequeñas, forzamos que el aprendizaje sea más lento, por lo que, con cada árbol nuevo, el modelo trata de corregir el error de forma más gradual. Esto hace que se necesiten muchos más árboles para acercarse al RMSE mínimo, lo cual se traduce en un coste computacional mucho mayor, pero a su vez, permite que se converja de forma más precisa.

Ademas, si tomamos una tasa de aprendizaje demasiado pequeña, cabe la posibilidad que se converja, no en el mínimo global, sino en un mínimo local. Sin embargo, en la práctica, no suele ser muy habitual que se converja en un mínimo local, y en general, cuanto menor sea la tasa de aprendizaje, mejores serán los resultados, siempre a costa de requerir bastante más tiempo para el entrenamiento del modelo.


+ __<font color=#5F66A1>max_depth</font>__:

Con este hiperparámetro restringimos la profundidad de cada árbol de decisión. Cuanto mayor sea la profundidad, más complejo será cada árbol de nuestro modelo, lo cual, si bien puede posibilitar predicciones más complicadas, también puede generar *overfitting*. 

A su vez, dado que a mayor profundidad, más complejidad, cuanto mayor sea nuestro valor de **max_depth**, por lo general menor será el número de árboles a partir del cual empezaremos a padecer un sobreentrenamiento.


+ __<font color=#5F66A1>min_child_weight</font>__:

En el caso de modelos de regresión, este hiperparámetro sirve para restringir el número de instancias de cada nodo, forzándolo a que sea siempre superior a **min_child_weight**. Es decir, si tras cierta partición del árbol, en alguno de los nodos que se han creado caen menos observaciones que las que se especifícan en el hiperparámetro, esa partición no se llevará a cabo.

Así, cuanto mayor sea su valor, más estaremos limitando la existencia de nodos muy específicos.
Y viceversa, a medida que su valor se vaya acercando a 1 (nodos sin ninguna restricción), más complejo será cada árbol, lo cual, nuevamente, si bien podría aumentar el poder predictivo del modelo, también puede generar *overfitting*.


+ __<font color=#5F66A1>subsample</font>__ & **<font color=#5F66A1>colsample_bytree</font>**: 

Con estos dos hiperparámetros restringimos el número de observaciones y variables para cada árbol. Sus valores van de 0 a 1, y reflejan el porcentaje de observaciones/columnas que se usan en la construcción de cada nuevo árbol, donde 0 significa que no se usa ninguna observación/columna, y donde 1 significa que se usan todas.

Esta restricción sirve para forzar a que cada árbol emplee distintas variables y distintas reglas para su construcción. Así, nuevamente, cuanto menor sea su valor, más conservadores seremos, lo cual podrá evitar un posible *overfitting* a costa de perder cierta complejidad.


+ __<font color=#5F66A1>gamma</font>__ & **<font color=#5F66A1>reg_alpha</font>**:

Ambos hiperparámetros sirven para regularizar nuestro hiperplano de regresión. Con **gamma** estableceremos el mínimo de reducción de nuestra función de coste para realizar una partición, y con **reg_alpha** estaremos añadiendo la regularización L1 (análoga a la regresión Lasso).

## Creación de funciones

A lo largo de la optimización de hiperparámetros, será necesario implementar y probar un gran número de modelos hasta encontrar aquel que mejores resultados nos ofrezca. Para esta tarea crearemos unas cuantas funciones:

La primeras dos funciones nos servirán para generar una búsqueda por rejilla y para plasmar los resultados de las distintas combinaciones mediante un mapa de calor. Así, mediante ambas funciones, trataremos de encontrar la mejor combinación de hiperparámetros posible:

In [ ]:
def grid_XGBRegressor(params, grid_p, X_train, y_train, cv_folds=5):
        
    gsearch = GridSearchCV(estimator = xgb.XGBRegressor(**params),
                               param_grid = grid_p,
                               scoring='neg_mean_squared_error',
                               n_jobs=8,
                               iid=False,
                               cv=cv_folds,
                               return_train_score=True,
                               verbose=True)
    
    gsearch.fit(X_train, y_train)
    
    return gsearch

In [ ]:
def plot_grid_XGBRegressor(gsearch, grid_p, grid_labels):
    
    x_len = len(grid_p[grid_labels[0]])
    x_label = grid_labels[0]
    
    if(len(grid_labels)==2):
        y_len = len(grid_p[grid_labels[1]])
        y_label = grid_labels[1]
    else:
        y_len = 1
        
                
    print(f'Los mejores parametros son: {gsearch.best_params_}\n')
    
    results_gsearch = pd.DataFrame(gsearch.cv_results_)
    scores_gsearch = np.array(np.sqrt(-results_gsearch.mean_test_score)).reshape(y_len, x_len)
    plt.figure(figsize=(12, y_len))

    if(len(grid_labels)==2):
        ax = sns.heatmap(scores_gsearch, cmap="rocket_r", annot=True, 
                 xticklabels=grid_p[x_label], yticklabels=grid_p[y_label], fmt='g')
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
    else:
        ax = sns.heatmap(scores_gsearch, cmap="rocket_r", annot=True, 
                 xticklabels=grid_p[x_label], fmt='g') 

La tercera la usaremos cuando queramos obtener un número de árboles adecuado aproximado mediante validación cruzada, valíendonos de *early stopping*:

In [ ]:
def get_n_estimators(params, dtrain, cv_folds=5, early_stopping_rounds=False, verbose=True):
    
    cvresult = xgb.cv(params, dtrain,               
                      num_boost_round=params['n_estimators'],
                      early_stopping_rounds=early_stopping_rounds,
                      nfold=cv_folds,
                      stratified=True,
                      metrics=params['eval_metric'],
                      seed=150,
                      verbose_eval=verbose)

    print(f'"n_estimators": {cvresult.shape[0]}')
    print(cvresult.tail(1))

La cuarta la usaremos para entrenar y validar nuestro modelo final:

In [ ]:
from sklearn.metrics import mean_squared_error

def xgb_fit_pred(params, dtrain, dtest, y_test):

    xgb_model = xgb.train(params, dtrain, num_boost_round=params['n_estimators'])
    y_pred = xgb_model.predict(dtest)    
    
    # Calculate eval metrics:
    residuals = y_pred - y_test
    SSD = np.sum(residuals ** 2)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred)) # RMSE = np.sqrt(SSD/len(y_pred)-len(X_train.columns)-1)
    error = 100*rmse/np.mean(y_test)
    
    # Plot residuals
    custome_residuals_plot(residuals, y_test, rmse)
    
    # Print model report:
    print(f"RMSE: {rmse}") # RMSE: Root Mean Squared Error:
    print(f"error: {round(error,2)}%")
    
    # Price prediction accuracy
    diff = pd.DataFrame(np.where(abs(residuals) < 15000, '<±15k',
        np.where(abs(residuals) < 30000, '±15k - ±30k',
        np.where(abs(residuals) < 50000, '±30k - ±50k',
        np.where(abs(residuals) < 75000, '±50k - ±75k','>±75k')))))

    results= pd.DataFrame()
    results['total'] = diff[0].value_counts()
    results['perc'] = round(100*results['total']/len(residuals),2)
    print(results['perc'])
    
    return xgb_model

Además, definimos una quinta función para visualizar los residuos entre el precio predicho y el precio real de las viviendas de nuestro modelo final, junto con el porcentaje de viviendas que están a un margen de distancia máximo del valor RMSE (raíz del error cuadrático medio):

In [ ]:
def custome_residuals_plot(residuals, y_test, rmse):
    
    f, axes = plt.subplots(figsize=(12, 6))
    sns.despine(left=True) 
    sns.scatterplot(y_test, residuals, color='#f8e9e2', alpha=.4, edgecolor='#e9b49e', linewidth=0.1)

    plt.plot((0, max(y_test)), (0,0),':', color='#636261')
    plt.plot((0, max(y_test)), (rmse, rmse), '-.', linewidth=0.4, color='#636261', alpha=0.5)
    plt.plot((0, max(y_test)), (-rmse, -rmse), '-.', linewidth=0.3, color='#636261', alpha=0.5)

    x_fill = np.arange(0,  max(y_test), max(y_test)/100)
    plt.fill_between(x_fill, -rmse, rmse, color='#636261', alpha=0.05)

    plt.text(max(y_test)/30, rmse*1.4, f'+{int(rmse)} €', color='#636261', size='large')
    plt.text(max(y_test)/30, -rmse*1.8 , f'-{int(rmse)} €', color='#636261', size='large')

    perc = round(np.where(abs(residuals) < rmse, 1,0).sum()*100/len(residuals),2)
    plt.text(max(y_test)*0.8, rmse*1.4, f'{perc}%', color='#636261', size='large', weight="bold")

    plt.title('\nRESIDUALS PLOT')
    plt.tight_layout()
    plt.show()

Por otro lado, trés cuatro funciones más, que las usaremos a la hora de visualizar las variables más importantes:

In [ ]:
def get_xgb_imp(xgb, feat_names):
    imp_vals = xgb.get_fscore()
    feats_imp = pd.DataFrame(imp_vals,index=np.arange(2)).T
    feats_imp.iloc[:,0]= feats_imp.index    
    feats_imp.columns=['feature','importance']
    feats_imp.sort_values('importance',inplace=True,ascending=False)
    feats_imp.reset_index(drop=True,inplace=True)
    
    return feats_imp

In [ ]:
def collapse_cat_importance(data, data2, param, prefix=False):
    
    if(prefix!=False):
        cats = [prefix + s for s in data2[param].unique().tolist()]
    else:
        cats = data2[param].unique().tolist()
        
    mask = data.feature.isin(cats)
    importance = data[mask].sum()[1]
    feats_collapsed = data[~mask]
    feats_collapsed = feats_collapsed.append({'feature' : param , 'importance' : importance} , ignore_index=True)
    
    return feats_collapsed.sort_values('importance', ascending=False)

In [ ]:
def custome_barplot(param1, param2, data, figW, figH, limit=50):

    f, axes = plt.subplots(figsize=(figW, figH))
    sns.despine(left=True)
    
    data[param1] = data[param1].apply(lambda x: x[:17])
    sns.barplot(x=param2, y=param1, data=data[:limit], color='#f8e9e2'
                , edgecolor='#e9b49e', linewidth=1.5, orient='h')
        
    max=0
    i=0
    for p in axes.patches:
        if max < p.get_width():
            max = p.get_width()
            
        plt.text(p.get_width()+max/100, p.get_y()+p.get_height()/2, f'{data[param2].iloc[i]}', color='#e18e7f', weight='bold', verticalalignment='center')
        i+=1
    plt.tight_layout()

Y por último, definimos otra función que nos servirá para reflejar la curva de aprendizaje de nuestro modelo:

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(model, X, y, cv=5, point_num=10):
    
    train_sizes, train_scores, test_scores = learning_curve(
        model, X, y, cv=cv, scoring="neg_mean_squared_error", train_sizes=np.linspace(0.01,1., point_num));

    train_error_mean = np.sqrt(-np.mean(train_scores, axis=1));
    test_error_mean = np.sqrt(-np.mean(test_scores, axis=1));

    fig, ax = plt.subplots(1, 1, figsize=(12,5))
    
    plt.plot(train_sizes, test_error_mean, 'o-', color='#e9b49e', label="Validation error")
    plt.plot(train_sizes, train_error_mean, 'o-', color="#636261", label="Train error")

    # .. plot error with text
    plt.text(train_sizes[0]+200,test_error_mean[0]-1000,int(test_error_mean[0]), color='#e9b49e')
    for i in range(len(test_error_mean)-1):
        plt.text(train_sizes[i+1]-300,test_error_mean[i+1]+2000,int(test_error_mean[i+1]), color='#e9b49e')
    for i in range(len(test_error_mean)):
        plt.text(train_sizes[i]-300,train_error_mean[i]+2000,int(train_error_mean[i]), color="#636261")

    plt.title(f"Curva de Aprendizaje")
    plt.xlabel("Número de muestras de entrenamiento")
    plt.ylabel("RMSE")
    plt.legend();

## Preparación de los datos

Antes de poder alimentar el modelo con datos, sera necesario adaptar estos de acuerdo con las necesidades del modelo. Por lo tanto, al ser un modelo de regresión, procedemos nuevamente a emplear **one-hot-encoding**, tal y como se hiciera en la implementación del modelo anterior:

No obstante, antes de nada realizaremos una criba preliminar de los datos que disponemos, descartando todas las viviendas con precio superior a los 400.000€:

In [ ]:
df4 = df.query('price < 400000')

dummy_houseType = pd.get_dummies(df4['house_type'])
dummy_garage = pd.get_dummies(df4['garage'], prefix='garage')
dummy_condition = pd.get_dummies(df4['condition'])
dummy_location = pd.get_dummies(df4['location'])

dummy_houseType = dummy_houseType.drop(columns=['Otros'])
dummy_garage = dummy_garage.drop(columns=['garage_sin garaje'])
dummy_condition = dummy_condition.drop(columns=['buen estado'])
dummy_location = dummy_location.iloc[:,1:]

df_dummy = (df4
            .join([dummy_houseType, dummy_garage, dummy_condition, dummy_location])
            .drop(columns=['house_type', 'garage', 'condition', 'location', 'loc_zone', 'loc_city']))

df_pred = df_dummy.drop(columns=['house_id', 'ad_description', 'pop_total', 'pop_>=65'])

## Implementación del modelo de regresión con XGBoost

### Division del conjunto de datos

El primer paso para implementar nuestro modelo, será dividir nuevamente nuestro conjunto de datos en 2 subconjuntos, uno de entrenamiento, y otro de test, en una proporción de 75% y 25% respectivamente. Por otro lado, separaremos también nuestra variable dependiente (X e y):

In [ ]:
df_pred.indoor = df_pred.indoor.astype('bool')
X = df_pred.drop(columns='price')
y = df_pred.price

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.25, random_state=89, shuffle=True)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

print(f'Número de datos para entrenamiento: {len(X_train)}')
print(f'Número de datos para test: {len(X_test)}')

### Modelo preliminar (parámetros por defecto)

Tras dividir el conjunto, entrenaremos y validaremos nuestro primer modelo XG-Boost sin realizar ningún tipo de hiperparametrización, empleando aquellos que se asignan por defecto. Esto nos valdrá para poder compararlo con nuestro modelo final, y  comprobar así si la estrategia de afinación de los hiperparámetros ha servido para mejorar el resultado preliminar:

In [ ]:
params = {
    'eval_metric': "rmse",
    'objective': "reg:squarederror",
    'n_estimators': 100,    
    'seed': 89,
    'nthread':8,
}

xgb_fit_pred(params, dtrain, dtest, y_test)

Vemos que el error es del 19.14%, con un 72,83% de los precios predichos dentro de un rango de +/- 40.895€ del precio real.

A partir de ahora, nuestra meta será realizar una afinación de los parámetros con la que mejorar estos resultados.

### max_depth & n_estimators

El primer paso de la hiperparametrización será hallar la mejor combinación para el número y la profundidad de los árboles. Ambos parámetros están estrechamente relacionados, dado que, por lo general, a mayor profundidad, menor es el número de árboles necesario. Por ello, emplearemos una rejilla con distintas combinaciones para así descubrir cual parece ser la mejor opción.

No obstante, empezaremos por redefinir el resto de parámetros, los cuales se mantendran constantes para cada combinación de la rejilla:

In [ ]:
upd = {
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'reg_alpha': 0,
    'learning_rate': 0.1,
}
params.update(upd)

Y a continuación, empleamos la búsqueda por rejilla (*Grid Search*) con una serie de valores para los parámetros **max_depth** y **n_estimators**: 

In [ ]:
grid_p = {
 'n_estimators': [200,400,600,800],
 'max_depth': [6,8,10,12],
}
grid_labels = ['n_estimators','max_depth']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Vemos que la mejor opción parece ser aquella que emplea 600 árboles con 6 niveles de profundidad. Por lo tanto, ajustemos la rejilla a este punto, y exploremos las combinaciones colindantes:

In [ ]:
grid_p = {
 'n_estimators': [450,550,650,750,850],
 'max_depth': [6,7,8,9],
}
grid_labels = ['n_estimators','max_depth']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

La mejor combinación es **n_estimators**=550 y **max_depth**=8. Por lo tanto, actualizamos nuestros parámetros con estos nuevos valores:

In [ ]:
upd = {
    'n_estimators': 550,
    'max_depth': 8
}
params.update(upd)

### min_child_weight

A continuación, trataremos de encontrar el valor de **min_child_weight** que mejor resultados parezca ofrecer. Para ello, nuevamente, emplearemos una búsqueda por rejilla:

In [ ]:
grid_p = {
 'min_child_weight': [1,2,3,4,5],
}
grid_labels = ['min_child_weight']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Vemos que el mejor valor para **min_child_weight** es el que ya teníamos (1), con lo cual, lo dejamos tal y como está.

### subsample & colsample_bytree

El próximo paso de nuestra hiperparametrización es la búsqueda de los mejores valores tanto para **subsample** como para **colsample_bytree**. Para ello, una vez más, nos valdremos de la búsqueda por rejilla:

In [ ]:
grid_p = {
    'subsample': [0.4,0.6,0.8,1],
    'colsample_bytree': [0.2,0.4,0.6,0.8]
}
grid_labels = ['subsample','colsample_bytree']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Volvemos a generar otra rejilla para acotar más los valores:

In [ ]:
grid_p = {
    'subsample': [0.7,0.75,0.8,0.85],
    'colsample_bytree': [0.35,0.4,0.45,0.5,0.55,0.6,0.65]
}
grid_labels = ['subsample','colsample_bytree']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Y volvemos nuevamente a generar otra tercera rejilla para afinar por última vez la búsqueda:

In [ ]:
grid_p = {
    'subsample': [0.78,0.80,0.82],
    'colsample_bytree': [0.48,0.5,0.52]
}
grid_labels = ['subsample','colsample_bytree']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Finalmente, actualizamos los parámetros con la mejor combinación de valores:

In [ ]:
upd = {
    'subsample': 0.8,
    'colsample_bytree':0.5,
}
params.update(upd)

### gamma & reg_alpha

Tras haber definido tanto el número como la profundidad y características de los árboles, así como el porcentaje de datos y columnas que emplearemos para cada uno de estos árboles, el siguiente paso es ajustar la regularización.

Para ello, nuevamente, nos valdremos de la búsqueda de rejilla para hallar, primero los mejores valores para **gamma**, y luego para  **reg_alpha**:

In [ ]:
grid_p = {
 'gamma': [0,1e-3,0.05,0.1,0.3,0.5,1,10],
}
grid_labels = ['gamma']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

A continuación, hacemos lo propio con **reg_alpha**:

In [ ]:
grid_p = {
 'reg_alpha': [0,1e-3,0.05,0.1,0.5,1,3,10],
}
grid_labels = ['reg_alpha']

gsearch = grid_XGBRegressor(params, grid_p, X_train, y_train)
plot_grid_XGBRegressor(gsearch, grid_p, grid_labels)

Vemos que la regularización no aporta ninguna mejora en este caso, por lo que dejamos ambos hiperparámetros tal y como están, con valores nulos.

### learning_rate & n_estimators

Una vez hemos seleccionado los valores para los distintos parámetros, el último paso será disminuir el valor de **learning_rate**, para así tratar de conseguir una afinación más precisa que, previsiblemente, hará que las predicciones sean más certeras.

No obstante, el escoger un **learning_rate** inferior acarreará que tengamos que escoger un nuevo número de árboles, ya que de mantener el que teníamos hasta ahora, no acabaría de converger lo suficiente (necesitará muchos más).

Por otro lado, cabe resaltar que los hiperparámetros que hemos ido seleccionando no son del todo independientes. Esto se traduce en que, el seleccionar nuevos valores tanto para **learning rate** como para **n_estimators**, probablemente, genere que la selección que hemos llevado a cabo en los pasos previos, si bien seguirá siendo buena, quizás ya no sea la óptima. Sin embargo, para poder descubir cuál es la nueva combinación óptima, tendríamos que haber realizado la búsqueda implementando modelos no con 550 árboles sino con muchísimos más, lo cual, hubiera supuesto tiempos de entrenamiento demasiado elevados.

Por lo tanto, nos limitaremos a mantener los parámetros seleccionados en los pasos previos, excepto el número de árboles, que lo actualizaremos valiéndonos de **early_stopping_rounds**: 

In [ ]:
upd = {
    'learning_rate': 0.01,
    'n_estimators': 20000,
}
params.update(upd)

get_n_estimators(params, dtrain, verbose=False, early_stopping_rounds=150, cv_folds=5)

In [ ]:
upd = {
    'n_estimators': 7000
}
params.update(upd)

xgb_model = xgb_fit_pred(params, dtrain, dtest, y_test)

Vemos que, tras la hiperparametrización, hemos conseguido reducir significativamente el RMSE de 40895 a 36717. Así, nuestro error ha pasado de un 19.14%, a un 17.18%, lo cual significa que la estrategia llevada a cabo en la hiperparametrización ha servido para disminuir de forma considerable nuestro error inicial.

De este modo, vemos que el modelo predice acertadamente el precio de la vivienda para el 74.83% de las viviendas con un margen de error cercano a los 37.000€.

Detallando más, vemos que el modelo predice correctamente el 41% de las viviendas con un margen de 15.000€, un 67% de las viviendas con un margen de 30.000€, y un 85% de las viviendas con un margen de 50.000€.

## Variables más relevantes

Una vez visto el poder predictivo de nuestro modelo, podemos aprovechar para comprobar cuáles han sido las variables mas importantes a la hora de predecir los precios de las viviendas, utilizando para ello los **F-Score**. Para ello, empezamos mostrando las variables más importantes en un gráfico:

In [ ]:
feature_importance_df = get_xgb_imp(xgb_model, X.columns.values)
custome_barplot('feature','importance',feature_importance_df,14,15)

En el gráfico superior, se observa que las distintas categorías de las variables categóricas se computan de forma independiente. Es decir, en lugar de computar la variable **location**, por ejemplo, se computa la importancia de cada una de las ubicaciones. Por ello, para medir, el impacto que tiene cada una de las variables categóricas, resulta más útil reflejar la importancia tras agrupar todas las categorías en una sola variable. Por lo tanto, se procede a ello:

In [ ]:
feature_importance_df = get_xgb_imp(xgb_model, X.columns.values)
feature_importance_df = collapse_cat_importance(feature_importance_df, df4, 'house_type')
feature_importance_df = collapse_cat_importance(feature_importance_df, df4, 'condition')
feature_importance_df = collapse_cat_importance(feature_importance_df, df4, 'location')
feature_importance_df = collapse_cat_importance(feature_importance_df, df4, 'garage', 'garage_')

custome_barplot('feature','importance',feature_importance_df,14,7)

Vemos que, ahora sí, las variables categóricas ya se reflejan de forma agrupada. Así, analizando la importancia de las variables, observamos lo siguiente...

...**el factor más relevante a la hora de predecir el precio de las viviendas es su tamaño**, definido en este caso mediante sus metros cuadrados reales (**m2_real**).

...tras el tamaño, vemos que la siguiente variable mas relevante es la ubicación de las viviendas (**location**).

... tras estas dos variables, las siguientes 3 variables más importantes son, en este orden, el número de planta de las viviendas (**floor**), la cantidad de habitaciones (**room_num**) y la cantidad de baños (**bath_num**).

De las cinco variables más relevantes, quizás, la única que sorprenda en cierto modo, sea la del número de planta de las viviendas, ya que, en el análisis exploratorio de la fase anterior, en principio, no parecía que el número de plantas pudiera tener un impacto directo en el precio, al menos no como para ser el tercer factor más determinante.

## Curva de aprendizaje

Por último, puede resultar interesante explorar si nuestro modelo podría beneficiarse de la obteniendo más datos. Para ello, reflejaremos la curva de aprendizaje de nuestro modelo final, dibujando así la evolución del error que genera nuestro modelo en función del número de muestras que usa para el entrenamiento:

In [ ]:
xgb_model_LC = xgb.XGBRegressor(**params)
%time plot_learning_curve(xgb_model_LC, X_train, y_train, cv=5, point_num=10)

Vemos por un lado que nuestro modelo tiene cierta varianza, ya que existe un espacio considerable entre la curva de aprendizaje de los datos de entrenamiento y la curva de aprendizaje de los datos de validación. Dicho de otro modo, nuestro modelo sufre un tanto de *overfiting*, haciendo que su rendimiento con los datos de entrenamiento sea bastante mejor que el rendimiento que ofrece con los datos de validacion. Por otro lado, atendiendo a la tendencia del error de validación, vemos que éste disminuye de forma gradual conforme se emplean más datos.

Por lo tanto, en vista de lo expuesto, **podemos intuir que la incorporación de más datos ayudaría a mejorar nuestro modelo**. Así, a fuerza de añadir más datos, el error de validación iría convergiendo cada vez más con el error de entrenamiento. Esta convergencia, probablemente, se diera en torno a los 30.000 RMSE, y a partir de ese punto, añadir más datos no nos serviría para reducir el error. Si quisieramos intentar reducir el RMSE por debajo de ese punto de convergencia, nuestro esfuerzo debería de centrarse, sobretodo, en tratar de incorporar no más datos sino otro tipo de variables distintas al estudio que ahora estemos pasando por alto.